In [ ]:
%load_ext autoreload
%autoreload 2

import os

import jax
jax.config.update("jax_enable_x64", True)

import numpyro
numpyro.set_host_device_count(4)

import numpy as np
import scipy.optimize

import ultraplot as uplt

In [ ]:
from des_y6_imsim_analysis import bump
from des_y6_imsim_analysis.stats import compute_model_chi2_info, measure_map, run_mcmc
from des_y6_imsim_analysis.utils import measure_m_dz, read_data

In [ ]:
fnames = [
    "../../data/N_gamma_alpha_v2_statistical.hdf5",
    # "../../data/N_gamma_alpha_v2_statistical-occupancy.hdf5",
    # "../../data/N_gamma_alpha_v2_statistical-neighbor.hdf5",
    # "../../data/N_gamma_alpha_v2_statistical-neighbor-occupancy.hdf5",
]

keys = [
    os.path.basename(fname).replace(".hdf5", "").split("_")[-1]
    for fname in fnames
]

seeds = [213432, 6457645, 213, 7878546]

fit_data = {}

model_module = bump

for key, fname, seed in zip(keys, fnames, seeds):
    data = read_data(fname)

    model_data = model_module.make_model_data(
        z=data.z,
        nzs=data.nzs,
        mn=data.mn,
        cov=data.cov,
        mn_pars=data.mn_pars,
        zbins=data.zbins,
        fixed_param_values={"w": 0.1, "g_b0": 0.0, "g_b1": 0.0, "g_b2": 0.0, "g_b3": 0.0},
        num_bins=8,
    )

    map_params = measure_map(
        model_module=model_module,
        model_data=model_data,
        seed=seed,
    )

    mcmc = run_mcmc(
        model_module=model_module,
        model_data=model_data,
        init_params=map_params,
        seed=seed,
    )

    chi2_info = compute_model_chi2_info(
        model_module=model_module,
        model_data=model_data,
        data=data,
        params=map_params,
    )

    fit_data[key] = {
        "map_params": map_params,
        "mcmc": mcmc,
        "model_data": model_data,
        "data": data,
    }

    print("key:", key)
    print("chi2:", chi2_info['chi2'])
    print("dof:", chi2_info['dof'])
    print("p-value:", chi2_info['p_value'])
    print("m vals:\n    model: %r\n    data: %r\n     err: %r" % (
        (chi2_info['model_mn'][:4] - 1).tolist(),
        (data.mn[:4] - 1).tolist(),
        (np.sqrt(np.diag(data.cov)[:4])).tolist(),
    ))
    print("", flush=True)

In [ ]:
rng = np.random.default_rng(75865874)

final_mdz = {}
n_per = 1000

for key in fit_data.keys():
    samples = fit_data[key]["mcmc"].get_samples()

    mdz = measure_m_dz(
        model_module=model_module,
        model_data=fit_data[key]["model_data"],
        samples=samples,
        return_dict=True,
    )
    n_samp = mdz[list(mdz.keys())[0]].shape[0]
    inds = rng.choice(n_samp, n_per, replace=False)
    for k, v in mdz.items():
        if k not in final_mdz:
            final_mdz[k] = []
        final_mdz[k].append(v[inds])

for k in final_mdz.keys():
    final_mdz[k] = np.concatenate(final_mdz[k], axis=0)

In [ ]:
m_mn = [np.mean(mdz[f"m_b{i}"]) for i in range(4)]
m_sd = [np.std(mdz[f"m_b{i}"]) for i in range(4)]

dz_mn = [np.mean(mdz[f"dz_b{i}"]) for i in range(4)]
dz_sd = [np.std(mdz[f"dz_b{i}"]) for i in range(4)]

print("|--------------------------------------------|")
print("| bin | m [10^-3, 3sigma] | dz               |")
print("|--------------------------------------------|")
for i in range(4):
    print(
        "| %d   | %+5.1f +/- %-5.1f   | %+0.3f +/- %0.3f |" % (
            i, m_mn[i]/1e-3, m_sd[i]/1e-3 * 3, dz_mn[i], dz_sd[i]
        )
    )
print("|--------------------------------------------|")

In [ ]:
arr = np.array(
    [mdz[f"m_b{i}"] for i in range(4)]
    + [mdz[f"dz_b{i}"] for i in range(4)]
)

In [ ]:
corr = np.corrcoef(arr)

In [ ]:
fig, axs = uplt.subplots(figsize=(5, 5))

m = axs.pcolormesh(np.abs(corr), vmin=0, vmax=1, cmap="gray_r", discrete=False)
axs.colorbar(m, label="Correlation Coefficient")
axs.format(grid=False)